# Modelling football stadium attendance using a GBM algorithm
The aim of this script is to generate an artificial data set to perform a GBM algorithm and deal with overfitting.
The script is slightly long, specially the data generation part, so do not hesitate to go fast through it. Thanks for you attention.

In [1]:
import h2o
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 min 17 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_a_nogue_sanchez_1p1ln9
H2O cluster total nodes:,1
H2O cluster free memory:,3.524 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


## Generating a random data set

In [2]:
random.seed(123)
sampleSize = 1000

In [3]:
# Generating stadium sizes (rounded at the closest hundred)
stadiumSize = []
for i in range (sampleSize):
    size = round(np.random.normal(loc = 50000, scale = 12000), -2)
    if(size < 10000):
        size = 10000
    stadiumSize.append(size)
#print(stadiumSize)

In [4]:
# Generating players' total value (rounded at the closest thousand)
playersValue = []
for i in range (sampleSize):
    # Hypothesis: players' value proportional to the stadium size + some randomness
    playersValue.append(round(stadiumSize[i] * 20 + random.randrange(200000, 500000, 1), -3))
#print(playersValue)

In [5]:
# Generating players' average age
playersAge = []
for i in range (sampleSize):
    # Hypothesis: age is completely random
    playersAge.append(round(np.random.normal(27, 2), 2))
#print(playersAge)

In [6]:
# Generating teams' winning average
victory = []
for i in range (sampleSize):
    # Hypothesis: winning 80% influenced by team's value + 20% of randomness
    percentage = round(playersValue[i]/np.max(playersValue)*80 + np.random.normal(loc=0.5, scale=0.1)*20)
    if(percentage < 0):
        percentage = 0
    elif(percentage > 100):
        percentage = 100
    victory.append(percentage)
#print(victory)

In [7]:
# Generating the output variable: average number of fans going to the stadium during the season
attendance = []
for i in range (sampleSize):
    # Hypothesis 1: attendance = stadium size * 0.8 +- some randomness
    v = round(np.random.normal(loc=stadiumSize[i]*0.6, scale = stadiumSize[i]*0.1), -2)
    # Hypothesis 2: the higher the players' value, the higher the attendance
    v = v + v*playersValue[i]/np.max(playersValue)*0.4
    # Hypothesis 3: the higher the victory rate, the higher the attendance
    v = v + v*victory[i]**0.5/100
    # Correcting for extreme values
    if(v < stadiumSize[i]*0.3):
        v = stadiumSize[i]*0.2
    elif(v > stadiumSize[i]*0.9):
        v = stadiumSize[i]*0.9
    attendance.append(round(v, -2))
#print(attendance)

In [8]:
# Creating the dataframe
teamsDF = pd.DataFrame(list(zip(stadiumSize, playersValue, playersAge, victory, attendance)))
teamsDF.columns =['StadiumSize','PlayersValue','PlayersAge','VictoryPercentage', "StadiumAttendance"]
print(teamsDF.head())  

hf = h2o.H2OFrame(teamsDF, destination_frame = "teams")

   StadiumSize  PlayersValue  PlayersAge  VictoryPercentage  StadiumAttendance
0      68900.0     1605000.0       24.15               69.0            62000.0
1      51800.0     1376000.0       25.27               58.0            42400.0
2      74200.0     1730000.0       28.82               74.0            64000.0
3      44400.0     1302000.0       27.05               60.0            19300.0
4      41400.0     1168000.0       28.31               52.0            24500.0
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
# Import the data
teams = h2o.get_frame("teams")

## Modelling a GBM algorithm

In [10]:
# Split the data into train and test set
train, test = teams.split_frame(
    ratios = [0.8],
    destination_frames = ["teams_train", "teams_test"],
    seed = 123)
train = h2o.get_frame("teams_train")
test = h2o.get_frame("teams_test")

In [11]:
# Defining X and Y variables
y = 'StadiumAttendance'
ignoreFields = y
x = [i for i in train.names if i not in ignoreFields]
print(x)

['StadiumSize', 'PlayersValue', 'PlayersAge', 'VictoryPercentage']


In [12]:
# Constructing a GBM algorithm (with default hyperparameters and no cross validation)
from h2o.estimators.gbm import H2OGradientBoostingEstimator
myGBM = H2OGradientBoostingEstimator(model_id = "baseline_GBM")
myGBM.train(x, y, train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [13]:
# Model performance
print("The GBM baseline model has a MAE of %d on the Train set and %d on the Test set"
      % (myGBM.mae(train), myGBM.model_performance(test).mae()))

The GBM baseline model has a MAE of 3248 on the Train set and 4360 on the Test set


The model exhibits better results in the train set than in the test set. Our model overfits.

In [14]:
# Completely overfitted model
myGBM_overfitted = H2OGradientBoostingEstimator(model_id = "overfitted_GBM", ntrees = 1000, max_depth = 10)
myGBM_overfitted.train(x, y, train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
print("The GBM overfitted model has a MAE of %d on the Train set and %d on the Test set"
      % (myGBM_overfitted.mae(train), myGBM_overfitted.model_performance(test).mae()))

The GBM overfitted model has a MAE of 208 on the Train set and 4864 on the Test set


We can see that the train error has significantly falled while the test error has increased. We can conclude that this specification indeed increases the previous overfitting.

In [16]:
# Constructing a final GBM: using cross validation to reduce overfitting
myGBM_cv = H2OGradientBoostingEstimator(model_id = "crossval_GBM", nfolds = 20)
myGBM_cv.train(x, y, train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
print("The GBM with cross validation model has a MAE of %d on the Train set and %d on the Test set"
      % (myGBM_cv.mae(train), myGBM_cv.model_performance(test).mae()))

The GBM with cross validation model has a MAE of 3248 on the Train set and 4360 on the Test set


The results exhibited by our CV model seem to be exactly the same as our baseline GBM. While using Cross Val, we train on different samples, so that the results are not supposed to be equal to a default GBM. I suppose I did a mistake somewhere.